There are two tasks that I need to complete for this week.

Firstly is building my own learning class/object/thingy.

Secondly I need to use this new learner object to create a model to classify the MNIST dataset.

Exciting!


In [1]:
# Needed modules

import torch
import numpy as np
from fastai.vision.all import *

from torch import nn

# built int

import os

## Building learner

A learner needs 4 bits of information to work
- Data
- Model
- Optimizer
- Loss function

### Getting data

In [387]:
class MyOptimizer:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            with torch.no_grad():
                p -= self.lr * p.grad

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

class MyLearner:
    def __init__(self, dls, model, loss_func, metric_func):
        self.dls = dls
        self.model = model
        self.loss_func = loss_func
        self.metric_func = metric_func

    def train_epoch(self, opt):
        updates = 0
        for xb, yb in self.dls.train:
            updates += 1
            pred = self.model(xb)
            loss = self.loss_func(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        print(f"Train loops: {updates}")
        return loss
    

    def validate_epoch(self):
        with torch.no_grad():
            accs = [self.metric_func(self.model(xb), yb) for xb,yb in self.dls.valid]
        return round(torch.stack(accs).mean().item(), 4) 

    
    def fit(self, epochs, lr):
        opt = MyOptimizer(self.model.parameters(), lr)
        opt.zero_grad()
        print("Epoch Train_Loss Valid_Accuracy  Time")
        for epoch in range(epochs):
            start = time.time()
            train_loss = self.train_epoch(opt)
            end = time.time()
            accuracy = self.validate_epoch()
            print(f" {epoch:02}:    {train_loss:.4f}     {accuracy:.4f}   {end-start:.4f}")



# 3 and 7 MNIST calssifier


## Getting data

In [196]:
path = untar_data(URLs.MNIST_SAMPLE)

### Training

In [197]:

threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]

stacked_3d = torch.stack(three_tensors).float()/255
stacked_7d = torch.stack(seven_tensors).float()/255

train_x = torch.cat([stacked_3d, stacked_7d]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)

dset = list(zip(train_x, train_y))
x,y = dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [209]:
dl = DataLoader(dset)

xb,yb = first(dl)
xb.shape, yb.shape

(torch.Size([784]), torch.Size([1]))

### Validation

In [211]:
valid_x = torch.cat([
    torch.stack([
        tensor(Image.open(o)).float()/255
        for o in (path/'valid'/'3').ls()
    ]),
    torch.stack([
        tensor(Image.open(o)).float()/255
        for o in (path/'valid'/'7').ls()
    ])
]).view(-1, 28*28)

valid_y = tensor([1]*len((path/'valid'/'3').ls()) + [0]*len((path/'valid'/'7').ls())).unsqueeze(1)

valid_dset = list(zip(valid_x, valid_y))
x,y = valid_dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [212]:
valid_dl = DataLoader(valid_dset)

xb,yb = first(valid_dl)
xb.shape, yb.shape

(torch.Size([784]), torch.Size([1]))

## Functions

In [213]:
def mnist_loss(preds, targets):
    preds = preds.sigmoid()
    return torch.where(targets==1, 1-preds, preds).mean()

mnist_loss(torch.tensor([0.9, 0.4, 0.2]), tensor([1,0,1]))

tensor(0.4460)

In [214]:
def mnist_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

mnist_accuracy(torch.tensor([0.9, 0.4, 0.2]), tensor([1,0,1]))

tensor(0.6667)

## Putting it all togather

In [215]:
dls = DataLoaders(dl, valid_dl)

In [216]:
dls.train.one_batch()[0].shape

torch.Size([784])

In [217]:
linear_model = nn.Linear(28*28, 1)

learner = MyLearner(dls, linear_model, mnist_loss, mnist_accuracy)

learner.fit(20, 0.001)

Epoch Train_Loss Valid_Accuracy  Time
Train loops: 12396
 00:    0.0027     0.5152   3.5418
Train loops: 12396
 01:    0.0008     0.8371   3.7759
Train loops: 12396
 02:    0.0004     0.9176   3.3921
Train loops: 12396
 03:    0.0003     0.9421   3.3636
Train loops: 12396
 04:    0.0002     0.9578   3.3978
Train loops: 12396
 05:    0.0002     0.9661   3.4027
Train loops: 12396
 06:    0.0002     0.9676   3.3492
Train loops: 12396
 07:    0.0002     0.9686   3.3554
Train loops: 12396
 08:    0.0001     0.9696   3.4631
Train loops: 12396
 09:    0.0001     0.9720   3.3660
Train loops: 12396
 10:    0.0001     0.9740   3.4234
Train loops: 12396
 11:    0.0001     0.9740   3.4455
Train loops: 12396
 12:    0.0001     0.9755   3.4930
Train loops: 12396
 13:    0.0001     0.9755   3.4971
Train loops: 12396
 14:    0.0001     0.9760   3.4730
Train loops: 12396
 15:    0.0001     0.9760   3.7055


KeyboardInterrupt: 

In [13]:
nn_basic = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
)

learner = MyLearner(dls, nn_basic, mnist_loss, mnist_accuracy)

learner.fit(20, 0.001)

Epoch Train_Loss Valid_Accuracy  Time
 00:    0.4790     0.5414   0.2384
 01:    0.4755     0.5839   0.1337
 02:    0.4719     0.6342   0.0825
 03:    0.4680     0.6914   0.0828
 04:    0.4638     0.7490   0.0874
 05:    0.4594     0.7969   0.1009
 06:    0.4548     0.8354   0.0771
 07:    0.4501     0.8598   0.0957
 08:    0.4452     0.8760   0.0939
 09:    0.4403     0.8906   0.0925
 10:    0.4351     0.9013   0.0875
 11:    0.4299     0.9145   0.0893
 12:    0.4246     0.9262   0.0778
 13:    0.4191     0.9331   0.0784
 14:    0.4136     0.9389   0.0746
 15:    0.4080     0.9453   0.0714
 16:    0.4022     0.9492   0.0818
 17:    0.3963     0.9516   0.0940
 18:    0.3902     0.9526   0.0819
 19:    0.3840     0.9521   0.0990


# Complete MNIST

## Getting data

In [223]:
path = untar_data(URLs.MNIST)
path.ls()

(#2) [Path('/home/james/.fastai/data/mnist_png/testing'),Path('/home/james/.fastai/data/mnist_png/training')]

In [224]:
train_path = (path/'training').ls().sorted()

all_data = [(tensor(Image.open(o)), int(path.name))
            for path in train_path
            for o in (path).ls()]

# shuffle

random.shuffle(all_data)


split = int(len(all_data)*0.8)

train_data = all_data[:split]
valid_data = all_data[split:]

len(train_data), len(valid_data)

(48000, 12000)

In [309]:
def get_dl(data_list):
    x = torch.stack([o[0] for o in data_list]).float()/255
    x = x.view(-1, 28*28)
    y = tensor([o[1] for o in data_list]).unsqueeze(1)
    print(x.shape, y.shape)

    dset = list(zip(x,y))

    print(len(dset))

    dl = DataLoader(dset, batch_size=64)

    return dl

train_dl = get_dl(train_data)
valid_dl = get_dl(valid_data)

train_dl.one_batch()[0].shape, valid_dl.one_batch()[1].shape

torch.Size([48000, 784]) torch.Size([48000, 1])
48000
torch.Size([12000, 784]) torch.Size([12000, 1])
12000


(torch.Size([64, 784]), torch.Size([64, 1]))

In [310]:
dls = DataLoaders(train_dl, valid_dl)

## Needed function

In [389]:
mnist_model = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 10)
)

mnist_model(train_dl.one_batch()[0]).shape

torch.Size([64, 10])

In [313]:
preds = mnist_model(train_dl.one_batch()[0])
targets = train_dl.one_batch()[1]

preds.shape, targets.shape

(torch.Size([64, 10]), torch.Size([64, 1]))

In [376]:
targets.view(-1)

tensor([6, 6, 8, 5, 6, 1, 8, 0, 1, 0, 8, 4, 2, 2, 8, 6, 3, 4, 4, 3, 2, 6, 7, 7,
        1, 3, 4, 8, 5, 3, 9, 4, 4, 1, 8, 3, 4, 4, 1, 2, 4, 1, 0, 6, 6, 7, 2, 9,
        7, 1, 7, 1, 4, 1, 8, 1, 7, 9, 2, 6, 6, 2, 7, 7])

In [378]:
def complete_mnist_loss(preds, targets):
    preds = preds.softmax(1)
    # display(preds)
    targets = targets.view(-1)
    # targets_probs = preds[range(targets.shape[0]), targets]

    # display(targets_probs)
    # return -torch.log(targets_probs).mean()

    # print(preds.shape, targets.shape)
    # print(targets * preds.log())
    # return -(targets * preds.log()).sum() / len(preds)
    return nn.functional.cross_entropy(preds, targets.squeeze())
    
random_preds = tensor([[0.9775, 0.7523, 0.7815, 0.5587, 0.9547, 0.3564, 0.9957, 0.5457, 0.9605,
         0.7544],
        [0.7494, 0.8851, 0.9722, 0.9944, 0.3382, 0.7579, 0.5753, 0.2673, 0.1166,
         0.3712],
        [0.0948, 0.6444, 0.0074, 0.1551, 0.0752, 0.8233, 0.2820, 0.0842, 0.5581,
         0.2289],
        [0.2603, 0.6603, 0.6184, 0.3055, 0.7156, 0.6018, 0.3889, 0.0617, 0.7188,
         0.6093],
        [0.6258, 0.9553, 0.1366, 0.4280, 0.8662, 0.4507, 0.7663, 0.3752, 0.7364,
         0.8468]])
display(random_preds)
random_targets = tensor([9, 5, 7, 4, 1])
display(random_targets)
complete_mnist_loss(random_preds, random_targets)

tensor([[0.9775, 0.7523, 0.7815, 0.5587, 0.9547, 0.3564, 0.9957, 0.5457, 0.9605,
         0.7544],
        [0.7494, 0.8851, 0.9722, 0.9944, 0.3382, 0.7579, 0.5753, 0.2673, 0.1166,
         0.3712],
        [0.0948, 0.6444, 0.0074, 0.1551, 0.0752, 0.8233, 0.2820, 0.0842, 0.5581,
         0.2289],
        [0.2603, 0.6603, 0.6184, 0.3055, 0.7156, 0.6018, 0.3889, 0.0617, 0.7188,
         0.6093],
        [0.6258, 0.9553, 0.1366, 0.4280, 0.8662, 0.4507, 0.7663, 0.3752, 0.7364,
         0.8468]])

tensor([9, 5, 7, 4, 1])

tensor(2.2939)

In [374]:
def complete_mnist_accuracy(preds, targets):
    preds = preds.argmax(dim=1)
    # display(preds)
    correct = (preds == targets).float()
    # display(correct)
    return correct.float().mean()

random_preds = torch.rand((5, 10))
display(random_preds)
random_targets = torch.randint(0, 10, (5,))
display(random_targets)
complete_mnist_accuracy(random_preds, random_targets)

tensor([[0.7238, 0.4907, 0.3476, 0.7817, 0.4462, 0.8248, 0.2465, 0.5707, 0.4818,
         0.0805],
        [0.5778, 0.6755, 0.9743, 0.2956, 0.1529, 0.9657, 0.5931, 0.7790, 0.3862,
         0.6850],
        [0.5436, 0.1356, 0.9448, 0.0588, 0.6334, 0.4536, 0.7372, 0.5188, 0.8084,
         0.1523],
        [0.0838, 0.2554, 0.2245, 0.7093, 0.4754, 0.6429, 0.8939, 0.0266, 0.2164,
         0.6699],
        [0.2741, 0.1632, 0.1362, 0.8435, 0.7617, 0.7495, 0.5814, 0.1199, 0.7856,
         0.2924]])

tensor([5, 0, 6, 7, 8])

tensor(0.2000)

In [392]:

learner = MyLearner(dls, mnist_model, complete_mnist_loss, complete_mnist_accuracy)

learner.fit(20, 0.001)

Epoch Train_Loss Valid_Accuracy  Time
Train loops: 750
 00:    2.2277     0.1031   1.0366
Train loops: 750
 01:    2.2201     0.1037   0.7042
Train loops: 750
 02:    2.2125     0.1043   0.6485
Train loops: 750
 03:    2.2045     0.1050   0.6578
Train loops: 750
 04:    2.1960     0.1056   0.8106
Train loops: 750
 05:    2.1870     0.1059   0.7406
Train loops: 750
 06:    2.1773     0.1064   0.7597
Train loops: 750
 07:    2.1669     0.1066   0.7072
Train loops: 750
 08:    2.1557     0.1066   0.6518
Train loops: 750
 09:    2.1436     0.1069   0.6202
Train loops: 750
 10:    2.1307     0.1072   0.6093
Train loops: 750
 11:    2.1169     0.1076   0.6077
Train loops: 750
 12:    2.1025     0.1080   0.6503
Train loops: 750
 13:    2.0877     0.1086   0.7099
Train loops: 750
 14:    2.0725     0.1092   0.7276
Train loops: 750
 15:    2.0571     0.1097   0.6900
Train loops: 750
 16:    2.0414     0.1100   0.6446
Train loops: 750
 17:    2.0257     0.1104   0.6531
Train loops: 750
 18:    2

In [343]:
fastai_learner = Learner(dls, mnist_model, mnist_loss, mnist_accuracy)

fastai_learner.fit(20, 0.001)

TypeError: mnist_accuracy() got an unexpected keyword argument 'lr'

In [318]:
mnist_model(train_dl.one_batch()[0]).argmax(dim=1)

tensor([8, 8, 8, 1, 4, 8, 1, 8, 4, 0, 4, 4, 4, 8, 4, 8, 1, 4, 4, 9, 9, 4, 4, 8,
        9, 8, 4, 8, 4, 9, 8, 4, 4, 8, 8, 1, 8, 8, 4, 9, 4, 4, 8, 8, 4, 8, 8, 8,
        8, 4, 8, 4, 8, 4, 4, 9, 1, 8, 8, 4, 4, 1, 4, 8])